In [ ]:
                           # Task - 1 ( preparing the data for segmentation)

In [ ]:
pip install tqdm

In [ ]:
# Import necessary libraries
import os
import cv2
import shutil
import numpy as np
import mahotas as mh
from PIL import Image
import matplotlib.pyplot as plt
from skimage import measure, filters, exposure



#new funtions added
from tqdm.notebook import tqdm
import functions as f

In [ ]:
# original directory path and working directory path for images
data = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir"
data_dir = data
raw_files = data + "/raw_files"
DATA_DIR = raw_files
good_images_mask = data + "/good_images/mask"
good_images_original = data + "/good_images/original"
npy_files = data_dir + "/npy_files"
temp_dir = data_dir + "/temp_dir"
cropped_dir = data_dir + "/cropped_images"
pre_dir = data_dir + "/pre_images"
print (data_dir)
print (raw_files)
print (good_images_mask)
print (good_images_original)
print (npy_files)
print (temp_dir)


In [ ]:
# original directory path and working directory path for images
data = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/raw_files"
working_dir = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir"

###########################################################################################
real = working_dir + "/test2/real_images"
normalized = working_dir + "/test2/normalized_images"
cropped = working_dir + "/test2/cropped_images"
npy = working_dir + "/test2/pre_processing/npy"
originals = working_dir + "/test2/pre_processing/originals"
masks = working_dir + "/test2/pre_processing/masks"
test = working_dir + "/test2/pre_processing/test"
s = working_dir + "/test2/S"

In [ ]:
# Create an empty list to store the images
slices = []
slice_address = []

In [ ]:
# check for any existing folders in the path, if there are any removing the old and creating new directories.

if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
            os.mkdir(temp_dir)
else:
            os.mkdir(temp_dir)

if os.path.exists(good_images_mask):
            shutil.rmtree(good_images_mask)
            os.mkdir(good_images_mask)
else:
            os.mkdir(good_images_original)

if os.path.exists(good_images_original):
            shutil.rmtree(good_images_original)
            os.mkdir(good_images_original)
else:
            os.mkdir(good_images_original)
        
if os.path.exists(npy_files):
            shutil.rmtree(npy_files)
            os.mkdir(npy_files)
else:
            os.mkdir(npy_files)
        
if os.path.exists(cropped_dir):
            shutil.rmtree(cropped_dir)
            os.mkdir(cropped_dir)
else:
            os.mkdir(cropped_dir)        


In [ ]:
#Copying the files from the RAW source to a temporary source for processing.
# added interactive progressbar to decrease the uncertanity and to increase curiosity :)
for z in tqdm(os.listdir(DATA_DIR)):
    # Check if the file ends with "tif"
    if z.endswith("tif"):
         shutil.copy(os.path.join(DATA_DIR, z), os.path.join(temp_dir, z))
            


In [ ]:
###############################NEEDS TO BE FIXED#################################
# #f.check(cropped_dir)
# def check(dir_path):
#     if os.path.isdir(dir_path):
#         #check(dir_path)
#         for file_name in os.listdir(dir_path):
#             file_path = os.path.join(dir_path, file_name)
#             file_size = os.path.getsize(file_path)
#             file_ctime = time.ctime(os.path.getctime(file_path))
#             print(f'{file_name} - Size: {file_size} bytes - Created: {file_ctime}')
#     else:
#         print(f"{dir_path} is not a valid directory")
#     check(cropped_dir)

In [ ]:
#####################################THIS ONE KIND OF APPROACH############
###############Normalize-crop approach#############################

In [ ]:
# Copying the files to cropped_dir vefore normalization so that temp_dir files won't be tampered and can
#be used as original images for training.           
for file_name in tqdm(sorted(os.listdir(temp_dir))):
    if file_name.endswith('.tif'):
        file_path = os.path.join(temp_dir, file_name)
        shutil.copy(file_path, cropped_dir)    

In [ ]:
f.normalize(cropped) #you can use this function.

# thinking of writing a code which will make a copy of 512*512 images after it breaks by every pixel by pixel move

# thinkin of writing a code which will make a copy of 512*512 images into multiple
# added interactive progressbar to decrease the uncertanity and to increase curiosity :)

# for z in tqdm(sorted(os.listdir(temp_dir))):
#     if (z.endswith("tif")): # checking the file ends with tif 
#         # Read in the image
#         img = mh.imread(os.path.join(temp_dir, z))

#         # Normalize the image
#         img = img.astype(np.float)
#         img /= img.max()
#         img *= 255

#         # Save the processed image back to the temporary directory
#         mh.imsave(os.path.join(temp_dir, z), img)

In [ ]:
# cropping images
#f.crop(temp_dir)
# thinking of writing a code which will make a copy of 512*512 images after it breaks by every pixel by pixel move

# thinkin of writing a code which will make a copy of 512*512 images into multiple
#For images in temporary folder reading them and cropping them to make it easy for segmentation.
def crop_images(cropped_dir):
    slices = []
    for z in tqdm(sorted(os.listdir(cropped_dir))):
        if z.endswith("tif"):
            # Read in the image
            img = mh.imread(os.path.join(cropped_dir, z))
            
            # Calculate the number of crops in each dimension
            height, width = img.shape[:2]
            num_crops_y = height // 512
            num_crops_x = width // 512
            
            for i in range(num_crops_y):
                for j in range(num_crops_x):
                    # Crop the image
                    start_y = i * 512
                    start_x = j * 512
                    img_cropped = img[start_y:start_y+512, start_x:start_x+512]
                    
                    # Create a new file name for the cropped image
                    file_name, file_ext = os.path.splitext(z)
                    new_file_name = f"{file_name}_{i}_{j}{file_ext}"
                    
                    # Copying temp images to cropped images folder
                    #shutil.copy(os.path.join(temp_dir, z), os.path.join(cropped_dir, new_file_name))
                    
                    # Save the processed image back to the cropped directory
                    mh.imsave(os.path.join(cropped_dir, new_file_name), img_cropped)
                    
                    # Add the image to the list
                    slices.append(img_cropped)


In [ ]:
crop_images(cropped_dir)

In [ ]:
crop_images(temp_dir)

In [ ]:
def count_files(dir_path):
    if os.path.isdir(dir_path):
        file_count = 0
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            file_size_bytes = os.path.getsize(file_path)
            file_size_mb = round(file_size_bytes / (1024 * 1024), 2)
            #print(f'{file_name} - Size: {file_size_mb} MB')
            file_count += 1
        return file_count
    else:
        print(f"{dir_path} is not a valid directory")
        return 0

In [ ]:
count_files(temp_dir)


In [ ]:
count_files(cropped_dir)

In [ ]:
pre_dir = data_dir + "/pre_images"

In [ ]:
# MASKING  Style -1

#from skimage import exposure

for z in tqdm(sorted(os.listdir(cropped_dir))):
    if (z.endswith("tif")): # checking the file ends with tif 
        img = mh.imread(os.path.join(cropped_dir, z))
    
    # Apply a Gaussian filter to the image
    b = img.copy()
    #b = mh.gaussian_filter(b, sigma=3)
    
    # Set values below 100 to 0
    b[b < 90] = 0
    
    
    #b = exposure.equalize_hist(b)
    # Label the regions in the filtered image
    labeled, number = mh.label(b)
    
    
    # filter based on labeled region size
    sizes = mh.labeled.labeled_size(labeled)
    
    # Remove the regions that are less than 1000
    too_small = np.where(sizes < 5000)
    labeled_only_big = mh.labeled.remove_regions(labeled, too_small)
    
    too_large = np.where(sizes > 20500)
    labeled_only_big = mh.labeled.remove_regions(labeled, too_large)
    #for debug
    #plt.imshow(labeled_only_big)
    #plt.show()
    
    
    # Create a binary mask from the filtered labeled regions
    binary_mask = labeled_only_big.copy()
    binary_mask[binary_mask > 0] = 1
    labeled, number_1 = mh.label(binary_mask)
    
    
     # Close the regions in the binary mask
    binary_mask_closed = mh.morph.close(binary_mask)
    plt.figure(figsize=(10,10))
    #plt.imshow(binary_mask_closed)
    #plt.show()    
    # Set a threshold for the minimum region size
    min_region_size = 3000
    # Initialize a variable to count the number of regions above the minimum size
    large_regions = 0
    
    # Get the sizes of the labeled regions
    region_sizes = measure.regionprops(labeled, intensity_image=binary_mask_closed)
        
    # Iterate over the region sizes and count the number of large regions
    for region in region_sizes:
        if region.area > min_region_size:
             large_regions += 1
                
    threshold = filters.threshold_otsu(binary_mask_closed) 
    binary_image = binary_mask_closed > threshold
    print(z)
    print("The image has clear segmentation.")
    plt.imshow(binary_image)
    plt.show()
    print (number_1)
    print (threshold)
    print(large_regions)
    print (sizes)
    print ("######################################################################")
    
   # if number_1 >=9500:
            #shutil.copy(os.path.join(cropped_dir, z), os.path.join(pre_dir, z))
            
             
#         mh.imsave(os.path.join(good_images_mask, z), binary_image)
        
#         shutil.copy(os.path.join(temp_dir, z), os.path.join(good_images_original, z))
        
        
        


In [ ]:
count_files(pre_dir)

In [ ]:
######################################THIS IS CROP AND NORMALIZE APPROACH###############################

In [ ]:
#For images in temporary folder reading them and cropping them to make it easy for segmentation.
for z in tqdm(sorted(os.listdir(temp_dir))):
    if (z.endswith("tif")): # checking the file ends with tif
        # Read in the image
        img = mh.imread(os.path.join(temp_dir, z))
        # Crop the image
        img_cropped = img[1500:2012, 2500:3012]
        #copying temp images to cropped images folder
        shutil.copy(os.path.join(temp_dir, z), os.path.join(cropped_dir, z))
        # Save the processed image back to the temporary directory
        mh.imsave(os.path.join(cropped_dir, z), img_cropped)
        # Add the image to the list
        slices.append(img_cropped)
        # Display the image
        print(z)
       # plt.imshow(img_cropped)
       # plt.show()

In [ ]:
for z in tqdm(sorted(os.listdir(cropped_dir))):
    if (z.endswith("tif")): # checking the file ends with tif 
        # Read in the image
        img = mh.imread(os.path.join(cropped_dir, z))

        # Normalize the image
        img = img.astype(np.float)
        img /= img.max()
        img *= 255

        # Save the processed image back to the temporary directory
        mh.imsave(os.path.join(cropped_dir, z), img)

In [ ]:
# Iterate over the images in the slices list
for z in tqdm(sorted(os.listdir(cropped_dir))):
    if (z.endswith("tif")): # checking the file ends with tif 
        img = mh.imread(os.path.join(cropped_dir, z))
    # Apply a Gaussian filter to the image
    b = img.copy()
    #b = mh.gaussian_filter(b, sigma=3)

    # Set values below 100 to 0
    b[b < 40] = 0

    # Label the regions in the filtered image
    labeled, number = mh.label(b)

    # filter based on labeled region size
    sizes = mh.labeled.labeled_size(labeled)
    
    print (sizes)

    # Remove the regions that are less than 1000
    too_small = np.where(sizes < 10)
    labeled_only_big = mh.labeled.remove_regions(labeled, too_small)

    # Create a binary mask from the filtered labeled regions
    binary_mask = labeled_only_big.copy()
    binary_mask[binary_mask > 0] = 1
    labeled, number_1 = mh.label(binary_mask)
    # Apply a Gaussian filter to the closed binary mask
    binary_mask_closed_filtered = mh.gaussian_filter(binary_mask_closed, sigma=3)
    # Check the number of labeled regions in the final binary mask
    labeled, number_final = mh.label(binary_mask_closed_filtered) 

    # Apply binary threshold to the image to segment the regions of interest
    threshold = filters.threshold_otsu(binary_mask_closed_filtered)
    binary_image = binary_mask_closed_filtered > threshold

    # Label the regions in the binary image
    labeled, num_regions = measure.label(binary_image, return_num=True)

    # Get the sizes of the labeled regions
    region_sizes = measure.regionprops(labeled, intensity_image=binary_mask_closed_filtered)

    # Set a threshold for the minimum region size
    min_region_size = 5000

    # Initialize a variable to count the number of regions above the minimum size
    large_regions = 0

    # Iterate over the region sizes and count the number of large regions
    for region in region_sizes:
        if region.area > min_region_size:
            large_regions += 1

    # Determine if the image has clear segmentation or not

    if number_1 <=50 and number_1 >=25:
        if large_regions <= 2:
            if threshold <= 0.46:
                
                print(z)
                print("The image has clear segmentation.")
               # plt.imshow(binary_image)
              #  plt.show()
                print (number_1, " <=50 and number_1 >=25 ")
                print (threshold, "<= 0.46")
                print(large_regions, ">= 2")
               # shutil.copy(os.path.join(temp, z), os.path.join(good, z))
               # mh.imsave(os.path.join(good, z), binary_image)
                print ("######################################################################")
            else:
                print (z)
               # plt.imshow(binary_image)
               # plt.show()
                print ("image diff -1")
                print (number_1, " <=50 and number_1 >=25 ")
                print (threshold,"<= 0.46")
                print(large_regions, ">= 2")
                #shutil.copy(os.path.join(temp, z), os.path.join(verybad, z))
               # mh.imsave(os.path.join(verybad, z), binary_image)
                print ("######################################################################")  
        else:
            print (z)
           # plt.imshow(binary_image)
           # plt.show()
            print ("image diff -2")
            print (number_1," <=50 and number_1 >=25 ")
            print (threshold,"<= 0.46")
            print(large_regions, ">= 2")
            #shutil.move(os.path.join(temp, z), os.path.join(verybad, z))
            #mh.imsave(os.path.join(verybad, z), binary_image)
            print ("######################################################################")    
    else:
        print (z)
       # plt.imshow(binary_image)
       # plt.show()
        print ("image diff -3")
        print (number_1, " <=50 and number_1 >=25 ")
        print (threshold, "<= 0.46")
        print(large_regions, ">= 2")
        #shutil.copy(os.path.join(temp, z), os.path.join(verybad, z))
        #mh.imsave(os.path.join(verybad, z), binary_image)
        print ("######################################################################") 
else:
    print("The image has a lot of noise in the segmentation.")
    print (z)
   # plt.imshow(binary_image)
   # plt.show()
    print (number_1, " <=50 and number_1 >=25 ")
    print (threshold, "<= 0.46")
    print(large_regions, ">= 2")
    #shutil.copy(os.path.join(temp, z), os.path.join(verybad, z))
    #mh.imsave(os.path.join(verybad, z), binary_image)    
    print ("######################################################################")
    

In [ ]:
#cropping images in good_original_ for NPY conversion

for z in tqdm(sorted(os.listdir(good_images_original))):
    if (z.endswith("tif")): # checking the file ends with tif
        # Read in the image
        img = mh.imread(os.path.join(good_images_original, z))
        # Crop the image
        img_cropped = img[1500:2012, 2500:3012]
            
        # Save the processed image back to the temporary directory
        mh.imsave(os.path.join(good_images_original, z), img_cropped)
        # Add the image to the list
        slices.append(img_cropped)
        # Display the image
        print(z)
     #   plt.imshow(img_cropped)
     #   plt.show()

In [ ]:
def plot_images(original_dir, mask_dir):
    num_images = len([f for f in os.listdir(original_dir) if f.endswith('.tif')])
    fig, axs = plt.subplots(num_images, 2, figsize=(5*2, 5*num_images))
    for i, z in enumerate(tqdm(sorted(os.listdir(original_dir)))):
        if z.endswith("tif"):
            # Read in the original image
            img_original = plt.imread(os.path.join(original_dir, z))
            # Read in the mask image
            img_mask = plt.imread(os.path.join(mask_dir, z))
            
            # Plot the original image on the left
            axs[i][0].imshow(img_original)
            axs[i][0].set_title(z)
            
            # Plot the mask image on the right
            axs[i][1].imshow(img_mask)
            axs[i][0].set_title(z)
    
   # plt.show()

In [ ]:
#plot_images(good_images_original,good_images_mask)

In [ ]:
#######################STEP-2  Creating NPY files for this data.

In [ ]:
# you can use 
#f.npyconversion(good_images_original,npy_files + '/original.npy' )
#f.npyconversion(good_images_mask, npy_files/mask.npy)
import numpy as np
from PIL import Image
import os



In [ ]:
def npyconversion(tif_dir, npy_path):
    tif_files = [f for f in os.listdir(tif_dir) if f.endswith('.tif')]
    data = []
    for tif_file in tqdm(tif_files):
        img = Image.open(os.path.join(tif_dir, tif_file))
        data.append(np.array(img))
    np.save(npy_path, data)


In [ ]:
npyconversion(good_images_original,npy_files + '/original.npy' )

In [ ]:
npyconversion(good_images_mask, npy_files + '/mask.npy')

In [ ]:
# i am cheching using check function to see the files created or not
#f.check(npy_files)

In [ ]:
############################step -3 setting up the U-net###############

In [ ]:
!pip install keras-unet

In [ ]:
import keras
from keras_unet.models import custom_unet

In [ ]:
images_file = os.path.join(npy_files, 'original.npy')
labels_file = os.path.join(npy_files, 'mask.npy')

In [ ]:
images = np.load(images_file)
labels = np.load(labels_file)

In [ ]:
images.shape

In [ ]:
# full dataset does not have the last channel
images = images.reshape(images.shape[0],images.shape[1],images.shape[2],1)
labels = labels.reshape(labels.shape[0],labels.shape[1],labels.shape[2],1)

In [ ]:
images.shape

In [ ]:
p = np.random.permutation(len(images))

In [ ]:
images = images[p]
labels = labels[p]

In [ ]:
labels = labels.astype(np.float64)

In [ ]:
images = images.astype(np.float64)

for i in range(images.shape[0]):
    images[i] = (images[i] - images[i].min()) / (images[i].max() - images[i].min()) # normalize individually


In [ ]:
images.shape

In [ ]:
X_train = images[0:50]
y_train = labels[0:50]
X_val = images[10:20]
y_val = labels[10:20]
X_test = images[0:]
y_test = labels[0:]

In [ ]:
model = custom_unet(
    input_shape=(512, 512, 1),
    use_batch_norm=False,
    num_classes=1,
    filters=32,
    dropout=0.5,
    output_activation='sigmoid')

In [ ]:
import keras.optimizers
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

In [ ]:
opt = keras.optimizer_v1.Adam(lr=0.01)

In [ ]:
model.compile(optimizer = 'Adam',    
              loss='binary_crossentropy', 
              metrics=[iou, iou_thresholded])

In [ ]:
history = model.fit(X_train, 
                    y_train, 
                    50, 
                    epochs=500,
                    validation_data=(X_val, y_val), 
                    verbose=1)

In [ ]:
from keras_unet.utils import plot_segm_history

plot_segm_history(history)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from keras_unet.utils import plot_imgs

plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=10)

In [ ]:
def process_image(dir):
    success = False  # Set default value for success
    success_count=0 
    for z in tqdm(sorted(os.listdir(dir))):
        if (z.endswith("tif")):            
            img=mh.imread(os.path.join(dir,z)) 
            print (z)
            a =img.shape
            plt.show()
            plt.imshow(img)           
            success = False            
            for a in range(100, 0, -1):
                b = img.copy()
                b = torch.from_numpy(b)
                # Move input data to GPU
                b = b.to('cuda')
                b.masked_fill_(b < a, 0)               
                b_np = b.cpu().numpy()
                labeled, number = scipy.ndimage.label(b_np)
                labeled = torch.from_numpy(labeled).to(b.device)                 
                number = labeled.max().item()                   
                sizes = torch.bincount(labeled.view(-1))             
                too_small = (sizes < 1500).nonzero(as_tuple=True)[0]
                labeled_only_big = labeled.clone()
                for i in too_small:
                    labeled_only_big[labeled == i] = 0
                binary_mask = (labeled_only_big > 0).float()
                number_1 = binary_mask.max().item()
                kernel_size=3
                kernel=torch.ones((kernel_size,kernel_size),device=binary_mask.device)
                binary_mask_closed=F.conv2d(binary_mask[None,None,...],kernel[None,None,...],padding=kernel_size//2)>0
                binary_mask_closed_filtered=(binary_mask_closed)
                binary_mask_closed_filtered = binary_mask_closed_filtered.to(torch.float32)                
                binary_mask_closed_filtered_np = binary_mask_closed_filtered.cpu().numpy()               
                labeled,number_final= label(binary_mask_closed_filtered_np)                                    
                threshold=binary_mask_closed_filtered.flatten().kthvalue(int(binary_mask_closed_filtered.numel()*0.5))[0]                     
                binary_image=binary_mask_closed_filtered>threshold              
                binary_image_np = binary_image.cpu().numpy()
                labeled,num_regions= label(binary_image_np)
                region_sizes=[(region==i+1).sum() for i in range(num_regions)]
                min_region_size=5000
                large_regions_counted=sum([size>min_region_size for size in region_sizes])
                for region in region_sizes:
                    if region.area > min_region_size:
                        large_regions_counted += 1
                        print (large_region_counted)
                if number_1 <=150 and number_1 >=100:
                    if large_regions_counted <=10:
                        success=True
                        #print(f'success: {success}')
                        percent_success=(success_count/len(os.listdir(cropped)))*100
                        print(f'percent_success: {percent_success}')
                        #print(f'large_regions_counted: {large_regions_counted}')
                        print ('number_1')
                        break
            if success==True:
            print (z)                    
            success_count+=1        
            percent_success=(success_count/len(os.listdir(dir)))*100        
            if percent_success>=50 and percent_success<100:
                 print('50% Success')                
            elif percent_success==100:
                 print('100% Success')                    
    # Move result back to CPU if needed
    result = result.to('cpu')
 

In [ ]:
import time

start_time = 0  # Define start_time in the global scope

def starttime():
    global start_time  # Use the global keyword to access the global start_time variable
    start_time = time.time()
    
def endtime():
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time:.2f} seconds")

In [ ]:
for i in range (5):
    starttime()

    a = ('Hello world')
    print (a)
    print ('iteration number',i)
    endtime()

In [ ]:
import time
import pynvml
for i in range (5):
    pynvml.nvmlInit()

    device_count = pynvml.nvmlDeviceGetCount()

    for i in range(device_count):
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        name = pynvml.nvmlDeviceGetName(handle)
        serial = pynvml.nvmlDeviceGetSerial(handle)
        util = pynvml.nvmlDeviceGetUtilizationRates(handle)
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        print(f"{name} - {serial} - Utilization: {util.gpu}% - Timestamp: {timestamp}")

    pynvml.nvmlShutdown()

In [ ]:
image_files = sorted([os.path.join(s,z) for z in os.listdir(s) if z.endswith("tif")])
for i in range(2):
            img=mh.imread(image_files[i])
            print (i)
            name_only = os.path.basename(image_files[i])
            print ("this iteration is with g value",i,"for the image :",name_only)
            #print(name_only)
            plt.imshow(img)
            plt.show() 
        
        
        